# Syntax examples

## Sample Data
Run the cell below to provide sample data for the examples in this section


In [0]:
DROP TABLE IF EXISTS diamonds;
CREATE TABLE IF NOT EXISTS diamonds (
    carat FLOAT,
    cut STRING,
    color STRING,
    clarity STRING
);

INSERT INTO diamonds (carat, cut, color, clarity) VALUES
(0.23, 'Ideal', 'E', 'SI2'),
(0.21, 'Premium', 'E', 'SI1'),
(0.23, 'Good', 'E', 'VS1'),
(0.29, 'Premium', 'I', 'VS2'),
(0.31, 'Good', 'J', 'SI2');

## Table comments

In [0]:
%sql
--note you will need a table called diamonds to run this
use default;
Create table more_diamonds
comment "This is a table with more diamonds"
as select * from diamonds

## UDF Example

In [0]:
--Create a function
create function if not exists plus_one(x int) 
returns integer
return x + 1;

--Invoke the function, note if the function is stored in a schema other than default you will need to specify the schema name when invoking the function
select plus_one(10);

# Arrays

## Sample Array Data
Run the cell below to provide sample data for the examples in this section

In [0]:
-- create array table faculty
drop table if exists faculty;
CREATE TABLE if not exists faculty (
    faculty_id INT,
    faculty_name VARCHAR(100),
    students ARRAY<STRUCT<student_id INT, total_no_of_courses INT>>
);

INSERT INTO faculty (faculty_id, faculty_name, students) VALUES
(1, 'Dr. Smith', ARRAY(STRUCT(101, 5), STRUCT(102, 3))),
(2, 'Dr. Johnson', ARRAY(STRUCT(103, 4), STRUCT(104, 2))),
(3, 'Dr. Williams', ARRAY(STRUCT(105, 6), STRUCT(106, 1)));

select * from faculty;

-- create array table sales
create or replace table sales as 
select 1 as batchId ,
	from_json('[{ "employeeId":1234,"sales" : 10000 },{ "employeeId":3232,"sales" : 30000 }]',
         'ARRAY<STRUCT<employeeId: BIGINT, sales: INT>>') as performance,
  current_timestamp() as insertDate
union all 
select 2 as batchId ,
  from_json('[{ "employeeId":1235,"sales" : 10500 },{ "employeeId":3233,"sales" : 32000 }]',
                'ARRAY<STRUCT<employeeId: BIGINT, sales: INT>>') as performance,
                current_timestamp() as insertDate;

select * from sales;                


## Filtering Arrays

In [0]:

--transform evaluutes the lambda function for each element in the array
select 
  faculty_id,
  students,
  transform(students, i -> i.total_no_of_courses <3) as student_low_enrolments
from faculty;

--the lambda function can also be written as a transformation
select 
  faculty_id,
  students,
  transform(students, i -> i.total_no_of_courses +1) as updated_enrolments
from faculty;

--filter returns only the elements which match the criteria
select 
  faculty_id,
  students,
  filter(students, i -> i.total_no_of_courses <3) as student_low_enrolments
from faculty;

## Exploding Arrays

In [0]:
--explode returns each elemnt as individual rows
select explode(performance) from sales;

--you can then run aggregations over the data
with CTE as (select explode (performance) as perfomance from sales)
select sum(CTE.perfomance.sales) from CTE;



## Nesting Array Functions

Links to definitions below, examples in next cell

[collect_list](https://docs.databricks.com/aws/en/sql/language-manual/functions/collect_list)

[flatten](https://docs.databricks.com/aws/en/sql/language-manual/functions/flatten)

[aggregate](https://docs.databricks.com/aws/en/sql/language-manual/functions/aggregate)


In [0]:
--another way to get same answer as the above cell would be this which avoids the need to pre-build the CTE
--before performing the aggregate on the CTE
select aggregate(flatten(collect_list(performance.sales)), 0, (x,y) -> x+y) as total_sales from sales;

--there's a lot going on here, lets break it down a bit...

--this just returns the sales node from the array
select performance.sales from sales;
--collect_list returns a single row array of arrays
select collect_list(performance.sales) from sales;
--flatten converts this to a single array
select flatten(collect_list(performance.sales)) from sales;
--aggregate is a function which takes an array and returns a single value
select aggregate(flatten(collect_list(performance.sales)), 0, (x,y) -> x+y) agg_sales from sales;